In [ ]:
from llm_more_better.model import RewardModelLM
from llm_more_better.data import get_anthropic_rlhf_data

In [ ]:
train_loader, val_loader, test_loader = get_anthropic_rlhf_data(
    batch_size=4,
    model_name="meta-llama/Llama-3.2-1B-Instruct",
)

In [ ]:
for batch in train_loader:
    break

In [ ]:
model = RewardModelLM("meta-llama/Llama-3.2-1B-Instruct")

In [ ]:
model.training_step(batch, 0)

In [ ]:
model(**batch["chosen_tokens"])